In [1]:
import json
import random
import os
import pickle
import time
import gc
import copy

from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import optim
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

In [4]:
processed_expose_train_unlabel_predict_list_path = "data/processed_train_expose_unlabel_predict_list.json"

doctype_encoder_path = 'model/doctype_encoder.pickle'

In [5]:
with open (doctype_encoder_path, 'rb') as doctype_encoder_file: 
    doctype_list =  pickle.load(doctype_encoder_file)
doctype_list, len(doctype_list)

(['作品分析',
  '治愈系文章',
  '情感解读',
  '行业解读',
  '科普知识文',
  '深度事件',
  '推荐文',
  '攻略文',
  '人物专栏',
  '物品评测'],
 10)

In [14]:
with open(processed_expose_train_unlabel_predict_list_path, "r", encoding='utf-8') as unlabel_predict_list_file:
    for line in tqdm(unlabel_predict_list_file):
        json_data = json.loads(line)
#         print(json_data)
        predict_index = np.argmax(json_data['predict_list'])
        predict_label = doctype_list[predict_index]
#         print(predict_label)
        max_predict_score = max(json_data['predict_list'])
        if max_predict_score < 0.5: 
            print(json_data)
            break

4it [00:00, ?it/s]

{'id': '0e7671cc-a98d-11eb-8239-7788095c0b0f', 'category': 1, 'doctype': '', 'paragraphs_num': 3, 'pic_num': 0, 'source': '', 'words_len': 739, 'text': '冲3连红，NBA预测：马刺VS鹈鹕，众星缺阵，鹈鹕迎来好机会。鹈鹕上一场比赛他们输给了雄鹿，英格拉姆错失绝平的机会被雄鹿复仇，鹈鹕前面6场比赛仅仅只赢了1场，但是最近的状态有所回暖，先后战胜了凯尔特人和活塞，上一场又和雄鹿打得有来有回，毫无疑问，进攻端的高效发挥是他们获胜的关键，而他们现在似乎有点放弃防守的感觉，放心和对手玩进攻大战，明天面对马刺残阵是个抢分的好机会，锡安几乎可以在内线为所欲为，马刺没人可以跟他对位，另外最近英格拉姆打马刺的锋线几乎也都是错位，所以马刺很难限制住明天鹈鹕的得分，考虑到马刺现在这个阵容得分能力又有限，连对阵雷霆仅仅得到了99分，因此，看好鹈鹕-4.5胜。凯尔特人118:112步行者，昨天分析也很简单，就是相信凯尔特人的天赋和球队文化，面对实力不如自己的步行者会迎来反弹获胜，今天也是如期的跑出，尽管杰伦布朗和塔图姆的双探花组合依旧表现低迷，但是肯巴沃克表现亮眼拿到了33分，另外汤普森和罗威以及蒂格都很高效，最终凭借着第三节一波流的确立了领先优势，有惊无险地拿下了步行者，步行者休息一周之后分别输给了勇士和凯尔特人，接下来他们还要经历非常艰苦的客场密集赛程，可以多多跟踪，烽火最近12中8，希望未来这十天有一个不错的收获，加油，回到主题，分析09:00马刺VS鹈鹕。', 'predict_list': [0.0011266764486208558, 0.01592712476849556, 0.0005940736155025661, 0.24539144337177277, 0.01184074766933918, 0.45812752842903137, 0.0005253106355667114, 0.2501547634601593, 0.007187999784946442, 0.009124280884861946]}


In [13]:
count = 0
with open(processed_expose_train_unlabel_predict_list_path, "r", encoding='utf-8') as unlabel_predict_list_file:
    for line in tqdm(unlabel_predict_list_file):
        json_data = json.loads(line)
#         print(json_data)
        predict_index = np.argmax(json_data['predict_list'])
        predict_label = doctype_list[predict_index]
#         print(predict_label)
        max_predict_score = max(json_data['predict_list'])
        if max_predict_score < 0.4: 
#             print(json_data)
#             break
            count += 1
count
# <0.4:10917 <0.5:39347 <0.6:85107

500000it [00:12, 41460.29it/s]


10917

In [15]:
with open(processed_expose_train_unlabel_predict_list_path, "r", encoding='utf-8') as unlabel_predict_list_file, \
     open("data/processed_train_expose_unlabel_predict_other_0.5.json", "w", encoding='utf-8') as unlabel_predict_other_file:
    for line in tqdm(unlabel_predict_list_file):
        json_data = json.loads(line)
        predict_index = np.argmax(json_data['predict_list'])
        predict_label = doctype_list[predict_index]
        max_predict_score = max(json_data['predict_list'])
        if max_predict_score < 0.5: 
            json_data['doctype'] = '其他'
            del json_data['predict_list']
            unlabel_predict_other_file.write(f"{json.dumps(json_data, ensure_ascii=False)}\n")

500000it [00:13, 37936.33it/s]


In [17]:
type([])==list

True